#Ví dụ 1

In [ ]:
import numpy as np

class MinNormSolver:
    MAX_ITER = 250
    STOP_CRIT = 1e-6

    def _min_norm_element_from2(v1v1, v1v2, v2v2):
        """
        Analytical solution for min_{c} |cx_1 + (1-c)x_2|_2^2
        d is the distance (objective) optimzed
        v1v1 = <x1,x1>
        v1v2 = <x1,x2>
        v2v2 = <x2,x2>
        """
        if v1v2 >= v1v1:
            # Case: Fig 1, third column
            gamma = 0.999
            cost = v1v1
            return gamma, cost
        if v1v2 >= v2v2:
            # Case: Fig 1, first column
            gamma = 0.001
            cost = v2v2
            return gamma, cost
        # Case: Fig 1, second column
        gamma = -1.0 * ( (v1v2 - v2v2) / (v1v1+v2v2 - 2*v1v2) )
        cost = v2v2 + gamma*(v1v2 - v2v2)
        return gamma, cost

    def _min_norm_2d(vecs, dps):
        """
        Find the minimum norm solution as combination of two points
        This solution is correct if vectors(gradients) lie in 2D
        ie. min_c |\sum c_i x_i|_2^2 st. \sum c_i = 1 , 1 >= c_1 >= 0 for all i, c_i + c_j = 1.0 for some i, j
        """
        dmin = 1e8
        for i in range(len(vecs)):
            for j in range(i+1,len(vecs)):
                if (i,j) not in dps:
                    dps[(i, j)] = 0.0
                    dps[(i,j)] = np.dot(vecs[i], vecs[j])
                    dps[(j, i)] = dps[(i, j)]
                if (i,i) not in dps:
                    dps[(i, i)] = 0.0
                    dps[(i,i)] = np.dot(vecs[i], vecs[i])
                if (j,j) not in dps:
                    dps[(j, j)] = 0.0
                    dps[(j, j)] = np.dot(vecs[j], vecs[j])
                c,d = MinNormSolver._min_norm_element_from2(dps[(i,i)], dps[(i,j)], dps[(j,j)])
                if d < dmin:
                    dmin = d
                    sol = [(i,j),c,d]
        return sol, dps

    def _projection2simplex(y):
        """
        Given y, it solves argmin_z |y-z|_2 st \sum z = 1 , 1 >= z_i >= 0 for all i
        """
        m = len(y)
        sorted_y = np.flip(np.sort(y), axis=0)
        tmpsum = 0.0
        tmax_f = (np.sum(y) - 1.0)/m
        for i in range(m-1):
            tmpsum+= sorted_y[i]
            tmax = (tmpsum - 1)/ (i+1.0)
            if tmax > sorted_y[i+1]:
                tmax_f = tmax
                break
        return np.maximum(y - tmax_f, np.zeros(y.shape))

    def _next_point(cur_val, grad, n):
        proj_grad = grad - ( np.sum(grad) / n )
        tm1 = -1.0*cur_val[proj_grad<0]/proj_grad[proj_grad<0]
        tm2 = (1.0 - cur_val[proj_grad>0])/(proj_grad[proj_grad>0])

        skippers = np.sum(tm1<1e-7) + np.sum(tm2<1e-7)
        t = 1
        if len(tm1[tm1>1e-7]) > 0:
            t = np.min(tm1[tm1>1e-7])
        if len(tm2[tm2>1e-7]) > 0:
            t = min(t, np.min(tm2[tm2>1e-7]))

        next_point = proj_grad*t + cur_val
        next_point = MinNormSolver._projection2simplex(next_point)
        return next_point

    def find_min_norm_element(vecs):
        """
        Given a list of vectors (vecs), this method finds the minimum norm element in the convex hull
        as min |u|_2 st. u = \sum c_i vecs[i] and \sum c_i = 1.
        It is quite geometric, and the main idea is the fact that if d_{ij} = min |u|_2 st u = c x_i + (1-c) x_j; the solution lies in (0, d_{i,j})
        Hence, we find the best 2-task solution, and then run the projected gradient descent until convergence
        """
        # Solution lying at the combination of two points
        dps = {}
        init_sol, dps = MinNormSolver._min_norm_2d(vecs, dps)

        n=len(vecs)
        sol_vec = np.zeros(n)
        sol_vec[init_sol[0][0]] = init_sol[1]
        sol_vec[init_sol[0][1]] = 1 - init_sol[1]

        if n < 3:
            # This is optimal for n=2, so return the solution
            return sol_vec , init_sol[2]

        iter_count = 0

        grad_mat = np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                grad_mat[i,j] = dps[(i, j)]

        while iter_count < MinNormSolver.MAX_ITER:
            grad_dir = -1.0*np.dot(grad_mat, sol_vec)
            new_point = MinNormSolver._next_point(sol_vec, grad_dir, n)
            # Re-compute the inner products for line search
            v1v1 = 0.0
            v1v2 = 0.0
            v2v2 = 0.0
            for i in range(n):
                for j in range(n):
                    v1v1 += sol_vec[i]*sol_vec[j]*dps[(i,j)]
                    v1v2 += sol_vec[i]*new_point[j]*dps[(i,j)]
                    v2v2 += new_point[i]*new_point[j]*dps[(i,j)]
            nc, nd = MinNormSolver._min_norm_element_from2(v1v1, v1v2, v2v2)
            new_sol_vec = nc*sol_vec + (1-nc)*new_point
            change = new_sol_vec - sol_vec
            if np.sum(np.abs(change)) < MinNormSolver.STOP_CRIT:
                return sol_vec, nd
            sol_vec = new_sol_vec
        return sol_vec, nd




#Ví dụ 2

In [ ]:
import numpy as np
from autograd import grad
import autograd.numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from math import e
from scipy.optimize import minimize


### the synthetic multi-objective problem ###
def f1(x):

    n = len(x)

    sum1 = np.sum([(x[i] - 1.0/np.sqrt(n)) ** 2 for i in range(n)])

    f1 = 1 - e**(- sum1)
    return f1

def f2(x):

    n = len(x)

    sum2 = np.sum([(x[i] + 1.0/np.sqrt(n)) ** 2 for i in range(n)])

    f2 = 1 - e**(- sum2)

    return f2

def f3(x):
    return f1(x)+f2(x)

def g1(x):
  return x[0]-1
def g2(x):
  return x[0]+1
def g3(x):
  return x[1]-1
def g4(x):
  return x[1]+1
def g5(x):
  return x[2]-1
def g6(x):
  return x[2]+1

f1_dx = grad(f1)
f2_dx = grad(f2)
f3_dx = grad(f3)
g1_df = grad(g1)
g2_df = grad(g2)
g3_df = grad(g3)
g4_df = grad(g4)
g5_df = grad(g5)
g6_df = grad(g6)

In [ ]:
import plotly.graph_objects as go
def concave_fun_eval(x):
    """
    return the function values and gradient values
    """
    return np.stack([f1(x), f2(x), f3(x)]), np.stack([f1_dx(x), f2_dx(x),f3_dx(x)])



### create the ground truth Pareto front ###
def create_pf_concave():

  ps = ps = np.linspace(-1/np.sqrt(2),1/np.sqrt(2), num = 100)

  pf = []

  for x1 in ps:
      #generate solutions on the Pareto front:
      x = np.array([x1,x1,x1])

      f, f_dx = concave_fun_eval(x)

      pf.append(f)

  pf = np.array(pf)

  return pf

pf = create_pf_concave()
fig = go.Figure(data=[go.Scatter3d(x = pf[:,0], y = pf[:,1], z = pf[:,2], mode='markers')])
fig.show()

In [ ]:
import autograd.numpy as np
from autograd import grad

from matplotlib import pyplot as plt

# use autograd to calculate the gradient
import autograd.numpy as np
from autograd import grad
from scipy.linalg import norm
from matplotlib import pyplot as plt

def get_d_paretomtl(grads,value, constraint,weights,i):
    # calculate the gradient direction for Pareto MTL
    nobj, dim = grads.shape

    # check active constraints
    normalized_current_weight = weights[i]/np.linalg.norm(weights[i])
    normalized_rest_weights = np.delete(weights, (i), axis=0) / np.linalg.norm(np.delete(weights, (i), axis=0), axis = 1,keepdims = True)
    w = normalized_rest_weights - normalized_current_weight

    # solve QP
    gx =  np.dot(w,value/np.linalg.norm(value))
    idx = gx >  0

    test = np.concatenate((grads, np.dot(w[idx],grads)), axis = 0)
    if test.ndim == constraint.ndim:
      vec =  np.concatenate((test, constraint), axis = 0)
    else:
      vec = test

    vec = vec/norm(vec)

    # use MinNormSolver to solve QP
    sol, nd = MinNormSolver.find_min_norm_element(vec)

    # reformulate ParetoMTL as linear scalarization method, return the weights
    weight0 =  sol[0] + np.sum(np.array([sol[j] * w[idx][j - 2,0] for j in np.arange(2,2 + np.sum(idx))]))
    weight1 = sol[1] + np.sum(np.array([sol[j] * w[idx][j - 2,1] for j in np.arange(2,2 + np.sum(idx))]))
    weight2 = sol[2] + np.sum(np.array([sol[j] * w[idx][j - 2,2] for j in np.arange(2,2 + np.sum(idx))]))

    num_cons = len(constraint)
    if num_cons != 0:
      weight_cons = sol[-num_cons:]
      weight = np.array([weight0] + [weight1] + [weight2] + weight_cons.tolist())
    else:
      weight = np.stack([weight0, weight1, weight2])
    return weight


def get_d_paretomtl_init(grads,value,constraint, weights,i):
    # calculate the gradient direction for Pareto MTL initialization
    nobj, dim = grads.shape

    # check active constraints
    normalized_current_weight = weights[i]/np.linalg.norm(weights[i])
    normalized_rest_weights = np.delete(weights, (i), axis=0) / np.linalg.norm(np.delete(weights, (i), axis=0), axis = 1,keepdims = True)
    w = normalized_rest_weights - normalized_current_weight

    gx =  np.dot(w,value/np.linalg.norm(value))
    idx = gx >  0

    if np.sum(idx) <= 0:
        return np.zeros(nobj)
    if np.sum(idx) == 1:
        sol = np.ones(1)
    else:
        test = np.dot(w[idx],grads)
        if constraint.ndim == test.ndim:
          vec =  np.concatenate((test, constraint), axis = 0)
        else:
          vec = test
        vec = vec/norm(vec)

        sol, nd = MinNormSolver.find_min_norm_element(vec)

    # calculate the weights
    weight0 =  np.sum(np.array([sol[j] * w[idx][j ,0] for j in np.arange(0, np.sum(idx))]))
    weight1 =  np.sum(np.array([sol[j] * w[idx][j ,1] for j in np.arange(0, np.sum(idx))]))
    weight2 =  np.sum(np.array([sol[j] * w[idx][j ,2] for j in np.arange(0, np.sum(idx))]))

    num_cons = len(constraint)
    if num_cons != 0:
      weight_cons = sol[-num_cons:]
      weight = np.array([weight0] + [weight1] +[weight2] + weight_cons.tolist())
    else:
      weight = np.stack([weight0, weight1, weight2])

    return weight


def circle_points(r, n):
    # generate evenly distributed preference vector
    circles = []
    for r, n in zip(r, n):
        t = np.linspace(0, np.pi, n)
        x = np.cos(t)*np.sin(t)
        y = np.sin(t)*np.sin(t)
        z = np.cos(t)

        x = r * x
        y = r * y
        z = r * z
        circles.append(np.c_[x, y, z])
    return circles

# def circle_points(r, n):
#     # generate evenly distributed preference vector
#     circles = []
#     for r, n in zip(r, n):
#         t = np.linspace(0, np.pi, n)
#         x = np.cos(t)
#         y = np.sin(t)
#         z = (np.full_like(t, 1) - x - y)
#         vector_sum = np.sum([x, y, z], axis=0)
#         x = r * x/vector_sum
#         y = r * y/vector_sum
#         z = r * z/vector_sum
#         circles.append(np.c_[x, y, z])
#     return circles

def circle_points2(K, min_angle=None, max_angle=None):
    # generate evenly distributed preference vector
    ang0 = np.pi / 30. if min_angle is None else min_angle
    ang1 = np.pi * 5 / 20. if max_angle is None else max_angle
    angles = np.linspace(ang0, ang1, K, endpoint=True)
    x = np.cos(angles)
    y = np.sin(angles)
    return np.c_[x, y]


# # calculate the gradients using autograd
# f1_dx = grad(f1)
# f2_dx = grad(f2)


def pareto_mtl_search_proposed(ref_vecs,i, t_iter = 500, n_dim = 3, step_size = 1, sigma = 1, kappa = 0.99, eps = 1e-20, count_check = 10):
    """
    Pareto MTL
    """
    count = 0
    # randomly generate one solution
    x = np.random.uniform(0, 1, n_dim)
    # x = find_min(x, n_dim)
    f_all = []
    x_all = []
    df_all = []

    # # find the initial solution
    for t in range(int(t_iter * 0.2)):
        f, f_dx = concave_fun_eval(x)
        # f, f_dx = convex_fun_eval(x)
        constraint = []
        value_set = [[g1(x),g1_df(x)], [g2(x),g2_df(x)], [g3(x),g3_df(x)], [g4(x),g4_df(x)],
                     [g5(x),g5_df(x)], [g6(x),g6_df(x)]]
        for f_value in value_set:
          if f_value[0]<=eps and f_value[0]>=-eps:
            constraint.append(f_value[1])
        constraint = np.array(constraint)
        weights =  get_d_paretomtl_init(f_dx,f, constraint, ref_vecs,i)

        if len(weights) > len(f_dx) and len(weights) - len(f_dx) == len(constraint):
          direction_descent = -np.dot(weights[0:len(f_dx)].T,f_dx).flatten() - np.dot(weights[len(f_dx):].T, constraint).flatten()
        else:
          if len(weights) != len(f_dx): break
          direction_descent = -np.dot(weights.T,f_dx).flatten()

        x = x + step_size * direction_descent
        # x = find_min(x, n_dim)
        x_all.append(x)

    # count = 0
    # find the Pareto optimal solution
    for t in range(int(t_iter * 0.8)):
        #f, f_dx = convex_fun_eval(x)
        f, f_dx = concave_fun_eval(x)
        f_all.append(f)
        df_all.append(f_dx)
        constraint = []
        value_set = [[g1(x),g1_df(x)], [g2(x),g2_df(x)], [g3(x),g3_df(x)], [g4(x),g4_df(x)],
                     [g5(x),g5_df(x)], [g6(x),g6_df(x)]]
        for f_value in value_set:
          if f_value[0]<=eps and f_value[0]>=-eps:
            constraint.append(f_value[1])
        constraint = np.array(constraint)

        weights =  get_d_paretomtl(f_dx,f,constraint, ref_vecs,i)

        if len(weights) > len(f_dx):
          direction_descent = -np.dot(weights[0:len(f_dx)].T,f_dx).flatten() - np.dot(weights[len(f_dx):].T, constraint).flatten()
          x = x + step_size * direction_descent  #x_k+1 = x_k + alpha*s_k
          #x_next = find_min(x_next, n_dim)
          f_after, f_dx_after = concave_fun_eval(x)
          #f_after, f_dx_after = convex_fun_eval(x_next)
          if np.all(f_after <= f + sigma*np.dot(f_dx, step_size * direction_descent)):
            step_size *= 1
          else:
            #print('Update')
            step_size *= kappa
            count+=1
        else:
          direction_descent = -np.dot(weights.T,f_dx).flatten()
          x = x + step_size * direction_descent  #x_k+1 = x_k + alpha*s_k
          #x_next = find_min(x_next, n_dim)
          f_after, f_dx_after = concave_fun_eval(x)
          #f_after, f_dx_after = convex_fun_eval(x_next)
          if np.all(f_after <= f + sigma*np.dot(f_dx, step_size * direction_descent)):
            step_size *= 1
          else:
            #print('Update')
            step_size *= kappa
            count+=1

        if norm(direction_descent) >= -eps and norm(direction_descent) <= eps:
          break

        # x = x + step_size * direction_descent
        #x = find_min(x, n_dim)
        x_all.append(x)
    # print(count)
    return x, f, x_all, f_all, df_all

In [ ]:
# pf = create_pf_concave()
f_value_list = []
x_value_list = []

num = 10

weights = circle_points([1], [num])[0]
#weights = circle_points2(num)

for i in range(num):

    print(i)

    x, f, x_all, f_all, df_all = pareto_mtl_search_proposed(ref_vecs = weights, i = i)
    print("gia tri f", f, " Gia tri x", x)
    f_value_list.append(f)
    x_value_list.append(x)

f_value_list = np.array(f_value_list)

0
gia tri f [7.02459235e-04 9.79649876e-01 9.80352336e-01]  Gia tri x [0.56204552 0.56204552 0.56204552]
1
gia tri f [0.96887661 0.01866428 0.98754089]  Gia tri x [-0.49810245 -0.49810245 -0.49810245]
2
gia tri f [0.24064347 0.88657164 1.1272151 ]  Gia tri x [0.27442901 0.27442901 0.27442901]
3


In [ ]:
fig = go.Figure()

scatter1 = go.Scatter3d(x = pf[:,0], y = pf[:,1], z = pf[:,2], mode='markers',
                        marker=dict(color='red', colorscale='Viridis', opacity=0.8), name = 'Pareto Front')
fig.add_trace(scatter1)

scatter2 = go.Scatter3d(x = f_value_list[:,0], y = f_value_list[:,1], z = f_value_list[:,2], mode='markers',
                        marker=dict(color='blue', colorscale='Hot', opacity=0.8), name = 'Solution')
fig.add_trace(scatter2)

fig.show()